In [1]:
# load data
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout, concatenate
from keras import backend as K
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras import regularizers


training_path = "../../Test_data/train.json"
testing_path = "../../Test_data/test.json"

train_data = pd.read_json(training_path)
test_data = pd.read_json(testing_path)


# no third for inc angle
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        # a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        # b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        # c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((band_1, band_2, band_3)))    # , c)))

    return np.array(imgs)

X_orig = get_scaled_imgs(train_data)

# resize to 150x150
X = []
for i in X_orig:
    X.append(cv2.resize(i, (150,150)))
X = np.array(X)

# get only data that has a non 0 inc angle
train_data.inc_angle = train_data.inc_angle.replace('na',0)
idx_tr = np.where(train_data.inc_angle>0)

Y = train_data["is_iceberg"]
inc_angle = train_data["inc_angle"]

Y_inc = Y[idx_tr[0]]
X_inc = X[idx_tr[0],...]
inc_angle_inc = inc_angle[idx_tr[0]]
# inc_angle_norm = (inc_angle_inc - inc_angle_inc.mean()) / (inc_angle_inc.max() - inc_angle_inc.min())


X_full = X_inc
Y_full = Y_inc
inc_full = inc_angle_inc

X_train, X_test, y_train, y_test, inc_train, inc_test = train_test_split(X_inc, Y_inc,
                                                    inc_angle_inc, test_size = 0.25, 
                                                                         random_state=42)

print("size x_train", len(X_train))
print("size x_test", len(X_test))
print("size inc train", len(inc_train))
print("size inc test", len(inc_test))
print("size y_train", len(y_train))
print("size y_test", len(y_test))

print("size x full", len(X_full))
print("size y full", len(Y_full))
print("size inc full", len(inc_full))




Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


size x_train 1103
size x_test 368
size inc train 1103
size inc test 368
size y_train 1103
size y_test 368
size x full 1471
size y full 1471
size inc full 1471


In [2]:
# this could also be the output a different Keras model or layer
from keras.layers import Flatten

input_tensor = Input(shape=(150, 150, 3))
ang_input = Input(shape=(1,))

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# add a global spatial average pooling layer

x = base_model.output
x = GlobalAveragePooling2D()(x)
# concat the angle
x = concatenate([x, ang_input])

# let's add a fully-connected layer
x = Dense(1024, kernel_regularizer=regularizers.l2(.1),
          bias_regularizer=regularizers.l2(.1))(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_regularizer=regularizers.l2(.1),
          bias_regularizer=regularizers.l2(.1))(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=[base_model.input ,ang_input], outputs=predictions)


# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

print("compiled")


compiled


In [3]:
# train the model on the new data for a few epochs, just to get the top layer going

# training code
#%% Image data augmentation 
from keras.preprocessing.image import ImageDataGenerator
import random


def generator_multiple(generator, X, inc_angle, y, batch_size):
    # shuffle initially (per epoch)
    shuff = list(zip(X, inc_angle, y))
    random.shuffle(shuff)
    X_shuff, inc_shuff, y_shuff = zip(*shuff)
    
    X_shuff = np.asarray(X_shuff)
    inc_shuff = np.asarray(inc_shuff)
    y_shuff = np.asarray(y_shuff)
    
        
    genX1 = generator.flow(X_shuff, y_shuff, shuffle=False, batch_size = batch_size)
    genX2 = generator.flow(X_shuff, inc_shuff, shuffle=False, batch_size = batch_size)
    
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        
        yield [X1i[0], X2i[1]], X1i[1]

datagen = ImageDataGenerator(
    featurewise_center=False,               # set input mean to 0 over the dataset
    samplewise_center=False,                # set each sample mean to 0
    featurewise_std_normalization=False,    # divide inputs by std of the dataset
    samplewise_std_normalization=False,     # divide each input by its std
    zca_whitening=False,                    # apply ZCA whitening
    rotation_range=0,                      # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.3,                  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.3,                 # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,                   # randomly flip images
    vertical_flip=False,                   # randomly flip images
    zoom_range=0.2)                     

datagen.fit(X_train)
model.fit_generator(generator_multiple(datagen, X_full, inc_full, Y_full, batch_size=32),
                    steps_per_epoch = len(X_full)/32.0, epochs = 20,
                     verbose= 1) #, validation_data = ([X_test,inc_test], y_test))   # validation_data = (X_test, y_test)

Epoch 1/20
46/45 [==============================] - 12s 257ms/step - loss: 237.9175 - acc: 0.4745
Epoch 2/20
46/45 [==============================] - 10s 225ms/step - loss: 233.0372 - acc: 0.4949
Epoch 3/20
46/45 [==============================] - 10s 227ms/step - loss: 228.3007 - acc: 0.5064
Epoch 4/20
46/45 [==============================] - 11s 238ms/step - loss: 223.7342 - acc: 0.5098
Epoch 5/20
46/45 [==============================] - 10s 227ms/step - loss: 219.1571 - acc: 0.5051
Epoch 6/20
46/45 [==============================] - 11s 239ms/step - loss: 214.6141 - acc: 0.5275
Epoch 7/20
46/45 [==============================] - 11s 237ms/step - loss: 210.1372 - acc: 0.5445
Epoch 8/20
46/45 [==============================] - 11s 233ms/step - loss: 205.7727 - acc: 0.5283
Epoch 9/20
46/45 [==============================] - 11s 233ms/step - loss: 201.5310 - acc: 0.5296
Epoch 10/20
46/45 [==============================] - 11s 234ms/step - loss: 197.2895 - acc: 0.5459
Epoch 11/20
46/45 [

In [ ]:

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)






In [4]:

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:1]:
   layer.trainable = False
for layer in model.layers[1:]:
   layer.trainable = True
# training them all
#for layer in model.layers:
#    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate

model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy',
              metrics=['accuracy'])

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = "model_weights_transf_inc_incV3.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=15)

print("recompiled")


recompiled


In [5]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
# train the model on the new data for a few epochs, just to get the top layer going

# training code
#%% Image data augmentation 
#from keras.preprocessing.image import ImageDataGenerator
#datagen = ImageDataGenerator(
#    featurewise_center=False,               # set input mean to 0 over the dataset
#    samplewise_center=False,                # set each sample mean to 0
#    featurewise_std_normalization=False,    # divide inputs by std of the dataset
#    samplewise_std_normalization=False,     # divide each input by its std
#    zca_whitening=False,                    # apply ZCA whitening
#    rotation_range=0,                      # randomly rotate images in the range (degrees, 0 to 180)
#    width_shift_range=0.3,                  # randomly shift images horizontally (fraction of total width)
#    height_shift_range=0.3,                 # randomly shift images vertically (fraction of total height)
#    horizontal_flip=False,                   # randomly flip images
#    vertical_flip=False,                   # randomly flip images
#    zoom_range=0.1) 

datagen.fit(X_train)
model.fit_generator(generator_multiple(datagen, X_full, inc_full, Y_full, batch_size=32),
                    steps_per_epoch = len(X_full)/32.0, epochs = 45,
                     verbose= 1) #, validation_data = ([X_test,inc_test], y_test))
                   

Epoch 1/45
46/45 [==============================] - 30s 648ms/step - loss: 154.8756 - acc: 0.5826
Epoch 2/45
46/45 [==============================] - 21s 466ms/step - loss: 150.9190 - acc: 0.6112
Epoch 3/45
46/45 [==============================] - 21s 465ms/step - loss: 146.9581 - acc: 0.6513
Epoch 4/45
46/45 [==============================] - 21s 466ms/step - loss: 143.1627 - acc: 0.6676
Epoch 5/45
46/45 [==============================] - 21s 466ms/step - loss: 139.4065 - acc: 0.6704
Epoch 6/45
46/45 [==============================] - 21s 465ms/step - loss: 135.7279 - acc: 0.6833
Epoch 7/45
46/45 [==============================] - 21s 466ms/step - loss: 132.1165 - acc: 0.7151
Epoch 8/45
46/45 [==============================] - 21s 465ms/step - loss: 128.5505 - acc: 0.7234
Epoch 9/45
46/45 [==============================] - 21s 466ms/step - loss: 125.0406 - acc: 0.7430
Epoch 10/45
46/45 [==============================] - 21s 467ms/step - loss: 121.6208 - acc: 0.7675
Epoch 11/45
46/45 [

In [6]:
# if doing no val set (i.e. final) only
filepath_full = 'model_weights_transf_inc_incV3_full.hdf5'
model.save(filepath_full)

In [7]:
# from above, need to see the amount of epochs (80 +) that val stops increasing at, then do 
# that in one full go and save the model

from keras.models import load_model
import pandas as pd
# if not using val
file_path_use = filepath_full
# if using val
#file_path_use = file_path
inf_model = load_model(file_path_use)

#score = inf_model.evaluate(X_test, y_test, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])


X_orig_test = get_scaled_imgs(test_data)

# resize to 150x150
X_sub = []
for i in X_orig_test:
    X_sub.append(cv2.resize(i, (150,150)))
X_sub = np.array(X_sub)
inc_angle_t = test_data["inc_angle"]


predicted_test=inf_model.predict([X_sub, inc_angle_t])

print("len of pred test", len(predicted_test))
print("len of id", len(test_data['id']))

submission = pd.DataFrame()
submission['id']=test_data['id']
submission['is_iceberg']=np.clip(predicted_test, 0.01, 0.99)
submission.to_csv('sub_full_transfer_inc_incV3.csv', index=False)

len of pred test 8424
len of id 8424
